In [ ]:
class Encoder():
    def __call__(self, inputs):
        """
        Args:
          inputs: [batch_size, num_nodes, node_dim] tensor
        Returns:
          [batch_size, num_nodes, embedding_dim] tensor
        """
        pass
    
class Decoder():
    def __call__(self, encoder_output, context_node_id, visited_mask):
        """Looks up embedding for context node, then does masked attention over encoder outputs.
        
        Result is log probs over next nodes.
        
        Args:
          inputs: [batch_size, num_nodes, embedding_dim]
          ...
          
        Returns:
          [batch_size, num_nodes] tensor of log probs with masking applied.
        """
        pass

class BatchedState():
    def __init__():
        """
        Should mirror the StateTSP of "attention learn how to solve routing problems"
        """

class BatchedGraphs():
    def __init__(self, edge_costs):
        """
        Args:
          edge_costs: [batch_size, num_edges, 2]
        """
        self._edge_costs = edge_costs
        
        
class BatchedQueues():
    def __init__(self, batched_graphs):
        pass
    
    def pop_batch(self):
        pass

In [ ]:
# TODO: generate random edge costs
num_nodes_per_graph = 20
num_batches = 100
batch_size = 10

# C++
graphs = BatchedGraphs(edge_costs)
graphs.pre_sort_edges()

# Python / GPU
encoder = Encoder()
decoder = Decoder()   
node_hiddens = Encoder(graphs)

# C++
queues = BatchedQueues(graphs)

# Python
for _ in range(num_batches):  # while queues not empty
    # C++
    current_nodes = queues.pop_batch()
    
    # C++ -> Python -- small amount of memory
    # current_nodes:
    #  * prefixes / visited masks
    #  * context nodes (TODO: what does that mean?)
    
    # CPU -> GPU
    # * current_nodes
    # NOT node_hiddens (should already be on GPU, so no cost to send in inner loop)
    
    # Python / GPU
    action_log_probs = decoder(node_hiddens, current_nodes)  # [batch, num_nodes] child log probs
    
    # ?? (probably python)
    special_actions = sample(action_log_probs)

    # GPU -> CPU
    # * special children [batch_size] of ints

    # ok to do on python CPU
    other_actions = get_other_actions(current_nodes, special_actions)
    
    # Convert actions to nodes
    # python or C++?
    
    # Python -> C++ -- no huge tensors
    # * batch of prefixes (though maybe this can live in C++ already)
    # * special children
    # * other children
    
    # C++
    # get batch of truncated gumbels
    # [batch] tensors of costs
    special_spanning_tree_costs = graphs.spanning_tree_cost(current_nodes.prefixes, special_children)
    # [batch] tensors of costs
    other_spanning_tree_costs = graphs.spanning_tree_cost(current_nodes.prefixes, other_children)
    
    # C++
    # missing: first node, next possible actions
    queues.add(special_children, special_spanning_tree_costs, special_gumbels)
    queues.add(other_children, other_spanning_tree_costs, other_gumbels)
   
    
    
    

In [1]:
import torch
import kruskals_cpp

In [ ]:
heap = kruskals_cpp.Heap()
node1 = kruskals_cpp.HeapNode(1.1, 3,torch.ones(10,1, dtype=torch.long))
heap.push(node1)
node2 = kruskals_cpp.HeapNode(3.4, 2, torch.zeros(10,1, dtype=torch.long))
heap.push(node2)
node3 = kruskals_cpp.HeapNode(3.4, 2,torch.ones(10,1, dtype=torch.long))
heap.push(node3)

In [7]:
parent = heap.pop()

In [9]:
parent.ids = torch.ones(2,2)

AttributeError: can't set attribute

In [5]:
torch.ones(2,2)

tensor([[1, 1],
        [1, 1]])